# Transform your samples posteriors to latex tables


@Author: P. Palma-Bifani

In [1]:
import glob
import numpy as np
import pandas as pd
import corner
import pickle

# my functions
from functions_df_to_latex import *

In [2]:
# test_file to check the functions and the workflow
path_test_file = '/Users/ppalmabifani/Desktop/exoAtm/c_ForMoSA/sub_modules/dF_to_latex/'

In [3]:

## INPUTS & format definitions
testdata_samples_direct = np.array([path_test_file + 'testdata.pic'] )         # give list with all the .pic files of all wanted models
bayesian_evidence       = True

# For ForMoSA the list of parameters availables is
# 'T_{eff}','logg','[M/H]','C/O', 'R_{jup}', 'RV', 'A_v', 'vsin(i)', '\epsilon', 'BB_{rad}', 'BB_{Teff}'
latex_list_param_name   = ['T_{eff}','logg','[M/H]','C/O']           # must be in same order as the sample file

## get tables
# For this case only header and 1 row
# Returns pandas dataframe and column names
latex_pd_dataframe, columns_latex   = latex_pandas_table(latex_list_param_name, testdata_samples_direct,evidencee=bayesian_evidence)


# SAVE PANDAS DATAFRAME DIRECTLY TO LATEX WITH THIS CODE-LINE
latex_pd_dataframe.to_csv(path_test_file+'testdata.txt', sep='&', line_terminator='\\\ \n', index=False, columns=columns_latex)



In [4]:
latex_pd_dataframe

,Index,T_{eff},logg,[M/H],C/O,log(z),h
0,0,$1551.8^{+228.5}_{-161.8}$,$3.24^{+0.67}_{-0.2}$,$0.49^{+0.27}_{-0.23}$,$0.72^{+0.07}_{-0.25}$,-1148.8 ± 0.4,17.2
